In [ ]:
import re, json, pathlib, logging, time, argparse, pprint
import datetime as dt
from itertools import chain
#
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import nltk as nltk
from nltk.corpus import stopwords
from whoosh.lang.porter import stem
#from googletrans import Translator
from pygoogletranslation import Translator

import polmap.polmap as plmp


##MM imports
#from polmap.polmap import make_directories, preprocess_text, doc2text, SDGrefs_mapper # replaced the keyword processing block

#import fromRtoPython

from docx2python import docx2python
 
#from polmap.polmap import make_directories, preprocess_text, doc2text # replaced the keyword processing block

In [ ]:
colors = ["#E5243B","#DDA63A","#4C9F38","#C5192D",
"#FF3A21","#26BDE2","#FCC30B","#A21942",
"#FD6925","#DD1367","#FD9D24","#BF8B2E",
"#3F7E44","#0A97D9","#56C02B","#00689D","#19486A"]

sdg_colors = {f'SDG {count.zfill(width)}' : color for count, color in enumerate(colors)}

bubble_chart = BubbleChart(area=total['Goal'],
                           bubble_spacing=0.1)

bubble_chart.collapse()

fig, ax = plt.subplots(subplot_kw=dict(aspect="equal"))
bubble_chart.plot(
    ax, browser_market_share['Goal'], sdg_colors[browser_market_share['Goal']])
ax.axis("off")
ax.relim()
ax.autoscale_view()
ax.set_title('Browser market share')

plt.show()

In [ ]:
#Expand keywords

input = 'keywords/keywords_copia.xlsx'
keywords = pd.ExcelFile(input)
filename = pathlib.Path(input).stem
timestamp =  dt.datetime.now().isoformat(timespec='seconds').replace(':','').replace('T','_T')
filename += f'_{timestamp}'

output = f'{filename}.xlsx'

#sheets = a list of sheets where keywords are
#keys_df = [keywords.parse(sheet) for sheet in sheets]
keys_dfs = [keywords.parse('Target_keys'), keywords.parse('Goal_keys'), keywords.parse('MOI')]

for index, df in enumerate(keys_dfs):
    df['Keys']=df['Keys'].apply(lambda keywords: re.sub(';$', '', keywords))
    df['Keys']=df['Keys'].apply(lambda keywords: re.sub(';;', ';', keywords))
    ex_df=df['Keys'].str.split(';', expand=True)
    lab_df=df.drop(columns='Keys')
    keys_dfs[index]=lab_df.join(ex_df)
    #df['Keys']=df['Keys'].apply(lambda keywords: re.sub(r'[^a-zA-Z0-9; -.]+', ';', keywords))

ex_df.head()

keys_dfs.append(keywords.parse('developing_countries'))

target_df, goal_df, devco_df, devco_names = keys_dfs

In [ ]:
target_df.head()

In [ ]:
ex_target_df=target_df['Keys'].str.split(';', expand=True)
lab_target_df=target_df.drop(columns='Keys')
lab_target_df.head()

In [ ]:
with pd.ExcelWriter(output, engine='xlsxwriter') as writer: # 'keywords_copy_for_transaltion.xlsx'
    target_df.to_excel(writer, sheet_name='Target_keys')
    goal_df.to_excel(writer, sheet_name='Goal_keys')
    devco_df.to_excel(writer, sheet_name='MOI')
    devco_names.to_excel(writer, sheet_name='developing_countries')

In [ ]:
#Load expand keywords

input = 'keywords/keywords_expanded.xlsx'

keywords = pd.ExcelFile(input)

sheets = keywords.sheet_names
print(f"Reading keywords dataset in {input}.\nSheets: {', '.join(sheets)}")

keywords = {sheet : keywords.parse(sheet, index_col=0) for sheet in sheets}
keywords_sheets = ['Target_keys', 'Goal_keys', 'MOI']

#remove all from stop_words to keep in keywords
stop_words = set(stopwords.words('english'))-set(['no','not','nor'])
stop_words.remove('all')

for sheet in keywords_sheets:
    # keywords[sheet]['Keys'] = keywords[sheet]['Keys'].apply(lambda keywords: re.sub(';$', '', keywords))
    # keywords[sheet]['Keys'] = keywords[sheet]['Keys'].apply(lambda keywords: [plmp.preprocess_text(keyword, stop_words) for keyword in keywords.split(';')])
    keywords[sheet].iloc[:,2:]=keywords[sheet].iloc[:,2:].applymap(lambda keyword: plmp.preprocess_text(str(keyword), stop_words))

In [ ]:
keywords['Target_keys'].head()

In [ ]:
#Load text

input = pathlib.Path('input/52019DC0640_stemmed')

files = sorted(input.glob('**/*.*'))

print(files)

text_ls = {}

for _file in files:
    with open(_file, 'r') as _f:
        text_ls[_file.stem]=_f.read()

In [ ]:
def count_keyword(text, keyword):

count = text.count(keyword)


In [ ]:
keywords['Target_keys']

In [ ]:
text_df = []

texts = list(text_ls.items())

keyword_matrix = keywords['Target_keys'].iloc[:,2:]#.to_numpy()
count_matrix = keywords['Target_keys'].iloc[:,2:].applymap(lambda keyword: texts[0].count(keyword))#.to_numpy()
labels = keywords['Target_keys'].iloc[:,0]


for name, text in text_ls.items():
    count_df = keywords['Target_keys'].iloc[:,2:].applymap(lambda keyword: text.count(keyword))
    #count_matrix = count_df#.to_numpy()
    count_df = count_df.join(labels, how='inner')
    count_df['Sum_counts'] = count_df.sum(axis=0)
    keywords_ = keyword_matrix[count_matrix>0]
    #count_df = count_df[count_df['Sum_counts'] > 0]
    # #df.replace(0, None, inplace=True)
    #df.insert(loc=1, column='Label', value=keywords['Target_keys'].iloc[:,1])
    # df.insert(loc=2, coulmn='Count_keys', value=df.apply(lambda x: sum(x),axis=1))
    # df.insert(loc=3, column='Diff_keys', value=df.apply(lambda x:count(x),axis=1))
    count_df.to_excel(f'test_{name}.xlsx')
    np.savetxt(f'test_{name}.csv', count_matrix, delimiter=',', fmt='%i')
    np.savetxt(f'test_{name}_keys.csv', keywords_, delimiter=',',fmt='%s')
    # np.savetxt('test.csv', count_matrix, delimiter=',')
    text_df.append(count_df)



In [ ]:
keyword_matrix.head()

In [ ]:
count_matrix.sum()